In [7]:
import tiktoken
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


In [8]:
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [11]:
loader = PyPDFLoader("/Users/jaehyeongpark/Documents/kb_report.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function = tiktoken_len)
docs = text_splitter.split_documents(pages)


model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
ko = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [12]:
db = FAISS.from_documents(docs, ko)

In [13]:
query = "주택 시장 동향은 어떻게 되어있나요?"
docs = db.similarity_search(query)
print(docs[0].page_content)

 서울과 수도권  아파트 를 중심으로  전세가격  상승세가  지속되고  있으며 , 수도권  전세가격전망지수는  
지난해  11월 이후 2개월 만에 다시 ‘상승 전망’으로 전환 
∙ 아파트  선호 현상과 올해 아파트  입주물량  감소 등의 영향으로  당분간  수도권  아파트  전세가
격 상승세가  지속될  전망 
 
그림 7| 유형별  주택 전세가격지수  변동률  추이  그림 8| 지역별  주택 전세가격전망지수  추이 
  
 
자료: KB국민은행   자료: KB국민은행


In [15]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores

[(Document(page_content='\uf06d 서울과 수도권  아파트 를 중심으로  전세가격  상승세가  지속되고  있으며 , 수도권  전세가격전망지수는  \n지난해  11월 이후 2개월 만에 다시 ‘상승 전망’으로 전환 \n∙ 아파트  선호 현상과 올해 아파트  입주물량  감소 등의 영향으로  당분간  수도권  아파트  전세가\n격 상승세가  지속될  전망 \n \n그림 7| 유형별  주택 전세가격지수  변동률  추이  그림 8| 지역별  주택 전세가격전망지수  추이 \n  \n \n자료: KB국민은행   자료: KB국민은행', metadata={'source': '/Users/jaehyeongpark/Documents/kb_report.pdf', 'page': 4}),
  0.75258374),
 (Document(page_content='\uf06d 1월 주택 매매가격은  전월 대비 하락폭이  확대되었으나  지난해  월평균  하락폭  대비 3분의 1 \n수준이며 , 하락폭이  가장 큰 대구도  지난해  월평균  하락폭  대비 절반 수준에 그침 \n\uf06d 주택 매매가격전망지수는  여전히  ‘상승 전망’보다 ‘하락 전망’ 비중이  크지만  연초 이후 정부의  규제 \n완화와  4월 총선, 금리 인하에 대한 기대감  등으로  하락폭이  확대될  가능성은  적음 \n \n그림 3| 지역별  주택 매매가격지수  변동률   그림 4| 지역별  주택 매매가격전망지수  추이 \n  \n \n자료: KB국민은행   자료: KB국민은행', metadata={'source': '/Users/jaehyeongpark/Documents/kb_report.pdf', 'page': 3}),
  0.75613165),
 (Document(page_content='5 \n[주택 전세] 수도권  아파트를 중심으로  전세가격  상승세가 지속될 전망 \n \n■ 주택 전세가격은 지난해  9월 이후 5개월 연속 상승세를 이어감  \n\uf06d 1월 전국 주택 전세가격 은 전

In [16]:
db.save_local("faiss_index")

In [25]:
new_db = FAISS.load_local("faiss_index", ko)

query = "주택 동향은?"
docs = new_db.similarity_search_with_relevance_scores(query, k=3, fetch_k=10, lambda_mult=0.3)


print("질문 : {} \n".format(query))
for i in range(len(docs)):
    print("{0}번쨰 유사 문서 유사도 \n{1}" .format(i+1, round(docs[i][1],2)))
    print("-"*100)
    print(docs[i][0].page_content)
    print("\n")
    print(docs[i][0].metadata)
    print("-"*100)

질문 : 주택 동향은? 

1번쨰 유사 문서 유사도 
0.43
----------------------------------------------------------------------------------------------------
 서울과 수도권  아파트 를 중심으로  전세가격  상승세가  지속되고  있으며 , 수도권  전세가격전망지수는  
지난해  11월 이후 2개월 만에 다시 ‘상승 전망’으로 전환 
∙ 아파트  선호 현상과 올해 아파트  입주물량  감소 등의 영향으로  당분간  수도권  아파트  전세가
격 상승세가  지속될  전망 
 
그림 7| 유형별  주택 전세가격지수  변동률  추이  그림 8| 지역별  주택 전세가격전망지수  추이 
  
 
자료: KB국민은행   자료: KB국민은행


{'source': '/Users/jaehyeongpark/Documents/kb_report.pdf', 'page': 4}
----------------------------------------------------------------------------------------------------
2번쨰 유사 문서 유사도 
0.4
----------------------------------------------------------------------------------------------------
자료: 한국부동산원   자료: 한국부동산원  
 
■ 매매 거래가  위축되면서  전월세  거래량은  최근 5년 평균 대비 21.7%  증가 
 1월 매수우 위지수는  소폭 상승했으나 , 역대 최저치를  기록했던  2022 년과 비슷한  수준을 유지 
 매매 수요가  위축되면서  전월세  수요가  증가하고  있으며 , 2023 년 전월세  거래량은  272만 건으로  
2년 연속 높은 수준을 유지 
∙ 2023 년 전월세  거래량은  전년 대비 4.1% 감소했으나  최근 5년 평균 대비 21.7%  증가 
 
그림 11| 지역별  주택 매